# Less Naive representation pipeline (pennylane)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [8]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
from pennylane.templates.embeddings import AmplitudeEmbedding

## Generating an artificial image of the sky

In [9]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[7.52049287e+01+3.92567195e+01j 7.52049287e+01+3.92567195e+01j
  2.10011972e-03+5.64683378e-03j 7.52049287e+01+3.92567195e+01j]
 [7.52049287e+01+3.92567195e+01j 5.59381000e-03+9.88065794e-03j
  4.81266261e-03+4.03659038e-03j 1.28731096e-03+4.97256302e-03j]
 [5.47536652e-03+5.55599580e-03j 4.80591316e-04+3.00519627e-03j
  9.92923345e-03+2.35517000e-03j 5.70890525e-03+5.70785346e-03j]
 [7.52049287e+01+3.92567195e+01j 1.81465751e-04+6.62067001e-03j
  6.49627203e-03+9.61707685e-03j 4.30099389e-03+2.42986853e-04j]]


## Applying 2D Fourier transform (visibilities)

In [10]:
visibilities = np.fft.fft2(sky_image)
print(visibilities)

[[ 376.07101004+196.3412392j   225.60550618+117.75910023j
    75.21618924 +39.25350126j  225.58833994+117.74901754j]
 [ 225.59770109+117.75846613j   75.20713953 +39.24242953j
   -75.2256137  -39.24618853j   75.21858627 +39.24994698j]
 [  75.20595036 +39.24362002j  -75.21416215 -39.24009658j
  -225.60341612-117.7572504j   -75.18618528 -39.25092715j]
 [ 225.59288293+117.75989623j   75.21283061 +39.24285757j
   -75.19847359 -39.25435307j   75.19057324 +39.25625337j]]


### Sanity check

In [11]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[7.52049287e+01+3.92567195e+01j 7.52049287e+01+3.92567195e+01j
  2.10011972e-03+5.64683378e-03j 7.52049287e+01+3.92567195e+01j]
 [7.52049287e+01+3.92567195e+01j 5.59381000e-03+9.88065794e-03j
  4.81266261e-03+4.03659038e-03j 1.28731096e-03+4.97256302e-03j]
 [5.47536652e-03+5.55599580e-03j 4.80591316e-04+3.00519627e-03j
  9.92923345e-03+2.35517000e-03j 5.70890525e-03+5.70785346e-03j]
 [7.52049287e+01+3.92567195e+01j 1.81465751e-04+6.62067001e-03j
  6.49627203e-03+9.61707685e-03j 4.30099389e-03+2.42986853e-04j]]

Reals MSE: 1.3994970470124195e-29
Imaginaries MSE: 1.2308736175238762e-29


## Classical data encoding/decoding

In [12]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
data_dimension = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(data_dimension))) # set the number of qbits (no padding needed if outputs an integer)
amp_dev = qml.device('default.qubit', wires, shots=1)

@qml.qnode(amp_dev)
def amp_encoding(data):
    AmplitudeEmbedding(data, wires) # normalize = True
    return qml.sample(qml.PauliX(0)), qml.sample(qml.PauliX(1)), qml.sample(qml.PauliX(2)), qml.sample(qml.PauliX(3))

amp_encoding(normalized_data)
readout = amp_dev.state

readout = readout*qml.math.sqrt(norm) # denormalization of the measurement outcomes
readout = np.array(readout).reshape(n , n)
print(readout)

[[ 376.07101004+196.3412392j   225.60550618+117.75910023j
    75.21618924 +39.25350126j  225.58833994+117.74901754j]
 [ 225.59770109+117.75846613j   75.20713953 +39.24242953j
   -75.2256137  -39.24618853j   75.21858627 +39.24994698j]
 [  75.20595036 +39.24362002j  -75.21416215 -39.24009658j
  -225.60341612-117.7572504j   -75.18618528 -39.25092715j]
 [ 225.59288293+117.75989623j   75.21283061 +39.24285757j
   -75.19847359 -39.25435307j   75.19057324 +39.25625337j]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [13]:
img = np.fft.ifft2(readout)

test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))


Reals MSE: 7.494740320319906e-29
Imaginaries MSE: 1.8811157149976167e-29


## Utils

### Quantum

### Classical

In [14]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask